# Swin-Tiny Training on Kaggle (Extracted Frames)

> This notebook trains on **extracted frame datasets** (no video decoding during training).

## Required Kaggle Datasets

1. **FF++ frames**:
   `/kaggle/input/datasets/muhammadqaiser1921/faceforenscis/ffpp_binary_frames`
   - splits: `train`, `val`, `test`
   - labels: `0=real`, `1=fake`

2. **Deepfake frames**:
   `/kaggle/input/datasets/aryansingh16/deepfake-dataset/real_vs_fake/real-vs-fake`
   - splits: `train`, `valid`, `test`
   - labels: `real`, `fake`

## Run Order

- **Cell 1**: Clone repo + verify datasets + install requirements
- **Cell 2**: Configure paths + train model
- **Cell 3**: View results + download outputs

In [ ]:
import os
import sys
import subprocess
import shutil

# ========== CONFIGURATION ==========
GITHUB_USERNAME = "MuhammadQaiser1921"
REPO_NAME = "swin-model"
REPO_BRANCH = "main"
GITHUB_URL = f"https://github.com/{GITHUB_USERNAME}/{REPO_NAME}.git"

FFPP_FRAMES_ROOT = "/kaggle/input/datasets/muhammadqaiser1921/faceforenscis/ffpp_binary_frames"
DEEPFAKE_FRAMES_ROOT = (
    "/kaggle/input/datasets/aryansingh16/deepfake-dataset/real_vs_fake/real-vs-fake"
 )

# ========== CLONE / UPDATE REPO ==========
os.chdir('/kaggle/working')
repo_path = os.path.join('/kaggle/working', REPO_NAME)

print(f"📌 Repository: {GITHUB_URL}")
print(f"🌿 Branch: {REPO_BRANCH}")
print(f"📁 Path: {repo_path}\n")

if os.path.exists(repo_path):
    try:
        os.chdir(repo_path)
        result = subprocess.run(["git", "status"], capture_output=True, text=True)
        if result.returncode == 0:
            print("✓ Using existing repo, fetching updates...")
            subprocess.run(["git", "fetch", "--all"], check=True)
            subprocess.run(["git", "checkout", REPO_BRANCH], check=True)
            subprocess.run(["git", "pull", "origin", REPO_BRANCH], check=True)
        else:
            os.chdir('/kaggle/working')
            print("⚠️ Invalid repo directory, removing and re-cloning...")
            shutil.rmtree(repo_path, ignore_errors=True)
            subprocess.run(["git", "clone", "-b", REPO_BRANCH, GITHUB_URL], check=True)
    except Exception as e:
        os.chdir('/kaggle/working')
        print(f"⚠️ Error: {e}. Removing and re-cloning...")
        shutil.rmtree(repo_path, ignore_errors=True)
        subprocess.run(["git", "clone", "-b", REPO_BRANCH, GITHUB_URL], check=True)
else:
    print(f"Cloning {GITHUB_URL}...")
    subprocess.run(["git", "clone", "-b", REPO_BRANCH, GITHUB_URL], check=True)

print("✅ Repository ready!\n")

# ========== VERIFY DATASETS ==========
def _check_path(path, label):
    if os.path.exists(path):
        print(f"✅ {label} found:")
        print(f"   {path}")
        print(f"   Top-level folders: {os.listdir(path)[:6]}")
    else:
        print(f"❌ {label} not found:")
        print(f"   {path}")

_check_path(FFPP_FRAMES_ROOT, "FF++ frames")
_check_path(DEEPFAKE_FRAMES_ROOT, "Deepfake frames")

# ========== OUTPUT DIRECTORIES ==========
os.makedirs(os.path.join(repo_path, 'models', 'checkpoints'), exist_ok=True)
os.makedirs(os.path.join(repo_path, 'models', 'weights'), exist_ok=True)
os.makedirs(os.path.join(repo_path, 'results', 'logs'), exist_ok=True)

# ========== INSTALL REQUIREMENTS ==========
sys.path.insert(0, os.path.join(repo_path, 'src'))
req_file = os.path.join(repo_path, 'requirements.txt')
if os.path.exists(req_file):
    print("Installing requirements...")
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "-r", req_file], check=True)
    print("✅ Requirements installed\n")
else:
    print("⚠️ requirements.txt not found\n")

In [ ]:
import tensorflow as tf
from train_video import train_video_model, Config

print("=" * 70)
print("🚀 TRAINING SWIN-TINY (EXTRACTED FRAMES)")
print("=" * 70)

# GPU check
gpus = tf.config.list_physical_devices('GPU')
print(f"\n💻 GPU available: {len(gpus)} GPU(s)")
for gpu in gpus:
    print(f"   ✓ {gpu}")

# Configure dataset roots
Config.FFPP_FRAMES_ROOT = "/kaggle/input/datasets/muhammadqaiser1921/faceforenscis/ffpp_binary_frames"
Config.DEEPFAKE_FRAMES_ROOT = (
    "/kaggle/input/datasets/aryansingh16/deepfake-dataset/real_vs_fake/real-vs-fake"
 )
Config.KAGGLE_ENV = True

# Training parameters
Config.BATCH_SIZE = 12
Config.EPOCHS = 20
Config.MAX_IMAGES_PER_CLASS = None  # Set to 500 for quick test

print("\n⚙️ Configuration:")
print(f"   FF++ root: {Config.FFPP_FRAMES_ROOT}")
print(f"   Deepfake root: {Config.DEEPFAKE_FRAMES_ROOT}")
print(f"   Batch size: {Config.BATCH_SIZE}")
print(f"   Epochs: {Config.EPOCHS}")
print(f"   Max images/class: {Config.MAX_IMAGES_PER_CLASS}")

print("\n🎯 Starting training...\n")
model, history, auc_metrics = train_video_model()

print("\n" + "=" * 70)
print("✅ TRAINING COMPLETED")
print("=" * 70)

if auc_metrics:
    print("\n📊 AUC METRICS:")
    print(f"   AUC-ROC: {auc_metrics.get('auc_roc', 'N/A'):.4f}")
    print(f"   PR-AUC: {auc_metrics.get('pr_auc', 'N/A'):.4f}")
    print(f"   Optimal Threshold: {auc_metrics.get('optimal_threshold', 'N/A'):.4f}")
    print(f"   Sensitivity: {auc_metrics.get('optimal_sensitivity', 'N/A'):.4f}")
    print(f"   Specificity: {auc_metrics.get('optimal_specificity', 'N/A'):.4f}")
    print(f"   F1-Score: {auc_metrics.get('optimal_f1', 'N/A'):.4f}")

print("\n💾 Outputs:")
print(f"   • {Config.CHECKPOINT_DIR}")
print(f"   • {Config.WEIGHTS_DIR}")
print(f"   • {Config.LOG_DIR}")

In [ ]:
import glob
import json
import os

print("\n" + "=" * 70)
print("📥 RESULTS & DOWNLOAD")
print("=" * 70)

results_dir = Config.LOG_DIR
weights_dir = Config.WEIGHTS_DIR
checkpoints_dir = Config.CHECKPOINT_DIR

print(f"\nLogs: {results_dir}")
print(f"Weights: {weights_dir}")
print(f"Checkpoints: {checkpoints_dir}")

# AUC metrics
auc_files = sorted(glob.glob(os.path.join(results_dir, 'auc_metrics_*.json')))
if auc_files:
    latest = auc_files[-1]
    print(f"\n✅ AUC metrics: {os.path.basename(latest)}")
    with open(latest, 'r') as f:
        metrics = json.load(f)
        print(f"   AUC-ROC: {metrics.get('auc_roc', 'N/A'):.4f}")
        print(f"   PR-AUC: {metrics.get('pr_auc', 'N/A'):.4f}")
else:
    print("\n⚠️ No AUC metrics found")

# Weights
weight_files = sorted(glob.glob(os.path.join(weights_dir, '*weights*.h5')))
if weight_files:
    print("\n✅ Weight files:")
    for path in weight_files[-3:]:
        size_mb = os.path.getsize(path) / (1024**2)
        print(f"   {os.path.basename(path)} ({size_mb:.1f} MB)")
else:
    print("\n⚠️ No weights found")

# Checkpoints
checkpoint_files = sorted(glob.glob(os.path.join(checkpoints_dir, '*.h5')))
if checkpoint_files:
    print("\n✅ Checkpoints:")
    for path in checkpoint_files[-2:]:
        size_mb = os.path.getsize(path) / (1024**2)
        print(f"   {os.path.basename(path)} ({size_mb:.1f} MB)")
else:
    print("\n⚠️ No checkpoints found")

print("\n📥 Download: Kaggle → Output → Download all")
print("=" * 70)